In [31]:
import matplotlib.axes as axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Exercício 1
owid_energy_data = pd.read_csv('owid-energy-data.csv')

countries = ['United States', 'Canada', 'Brazil', 'Mexico']
filtered_index = owid_energy_data.country.isin(countries)
energy_data = owid_energy_data[filtered_index]
energy_data.to_csv('filtered-energy-data.csv')

In [ ]:
# Exercício 2
plt.figure()
colors = ['blue', 'red', 'green', 'yellow']

for country, color in zip(countries, colors):
    data = energy_data[(energy_data.country == country)
                       & (energy_data['oil_electricity'].notnull())]
    plt.plot(data.year, data['oil_electricity'], '-', color=color, label=country)

plt.xlabel('year')
plt.ylabel('oil electricity')
plt.title('Electricity production from oil in certain countries')
plt.legend()
plt.show()

In [ ]:
# Exercício 3
usa_energy_data = energy_data[(energy_data.country == 'United States')
                              & (energy_data.year == 2010)]

elec_sources = ['coal_electricity', 'biofuel_electricity',
                'fossil_electricity', 'gas_electricity',
                'hydro_electricity', 'nuclear_electricity',
                'oil_electricity']

usa_electricity = usa_energy_data[elec_sources].values.flatten().tolist()

plt.figure()
plt.title('Electricity production from the United States in 2010')
elec_sources = [s.replace('_', ' ').capitalize() for s in elec_sources]
plt.pie(usa_electricity, labels=elec_sources)
plt.show()

In [32]:
# Exercício 4
def highest_nuclear_consumption(country: str) -> np.float64:
    country_data: pd.DataFrame = energy_data.loc[(energy_data.country == country),
                                                 ['year', 'nuclear_consumption']]
    i = country_data['nuclear_consumption'].idxmax()
    year = country_data.loc[i, 'year']
    consumption = country_data.loc[i, 'nuclear_consumption']
    print(f'Nuclear consumption of {country} in {year} was {consumption}')
    return consumption

In [ ]:
print('Highest nuclear consumption year per country:')
[highest_nuclear_consumption(country) for country in countries]

In [ ]:
# Exercício 5
sns.regplot(data=owid_energy_data, x='gas_consumption', y='nuclear_consumption')

plt.xlabel('Gas consumption')
plt.ylabel('Nuclear consumption')
plt.title(f'Gas consumption / Nuclear consumption')

grid = sns.FacetGrid(energy_data, col="country", hue="country", col_wrap=2,
                     sharex=False, sharey=False)
grid.map(sns.regplot, 'gas_consumption', 'nuclear_consumption')
grid.add_legend()
plt.show()

In [58]:
# Exercício 6
from warnings import simplefilter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'

def plot_multistep(y: pd.DataFrame, every: int = 1, ax: axes.Axes = None, palette_kwargs: dict = None):
    palette_kwargs_ = dict(palette='husl', n_colors=16, desat=None)
    if palette_kwargs is not None:
        palette_kwargs_.update(palette_kwargs)
    palette = sns.color_palette(**palette_kwargs_)
    if ax is None:
        _, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', palette))
    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods=len(preds), freq='Y')
        preds.plot(ax=ax)
    return ax

In [ ]:
# Exercício 6
solar_data: pd.DataFrame = energy_data.loc[(energy_data.country == countries[0]) & (energy_data.solar_consumption.notnull()),
                                           ['year', 'solar_consumption']]
solar_data.set_index('year', inplace=True, drop=True)

In [5]:
def make_lags(ts: pd.DataFrame, lags: int, lead_time: int = 1) -> pd.DataFrame:
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)

def make_multistep_target(ts: pd.DataFrame, steps: int):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)

In [ ]:
y = solar_data.solar_consumption.copy()
X = make_lags(y, lags=2).fillna(0.0)

# 4-years forecast
y = make_multistep_target(y, steps=4).dropna()

# Shifting has created indexes that don't match. Only keep times for
# which we have both targets and features.
y, X = y.align(X, join='inner', axis=0)

In [49]:
from sklearn.multioutput import MultiOutputRegressor

# Create splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

model = MultiOutputRegressor(XGBRegressor())
model.fit(X_train, y_train)

y_fit = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=y.columns)
y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)

In [ ]:
# Plotting
train_rmse = mean_squared_error(y_train, y_fit, squared=False)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Train RMSE: {train_rmse:.2f}\n" f"Test RMSE: {test_rmse:.2f}")

palette = dict(palette='husl', n_colors=64)
_, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 6))
ax1 = solar_data.solar_consumption[y_fit.index].plot(**plot_params, ax=ax1)
ax1 = plot_multistep(y_fit, ax=ax1, palette_kwargs=palette)
_ = ax1.legend(['Solar consumption (train)', 'Forecast'])
ax2 = solar_data.solar_consumption[y_pred.index].plot(**plot_params, ax=ax2)
ax2 = plot_multistep(y_pred, ax=ax2, palette_kwargs=palette)
_ = ax2.legend(['Solar consumption (test)', 'Forecast'])